In [1]:
import tweepy
import pandas as pd
import numpy as np
from config import authenticate_user

auth = authenticate_user()

api = tweepy.API(auth)

In [2]:
#setting user id, this is @elonmusk
id="44196397"

#setting initial full list
full_list = api.user_timeline(user_id=id,count=3200,exclude_replies=True,include_rts=False)

#saving max id
currmax = full_list[len(full_list)-1].id

#pop last id so no overlap
full_list.pop()

#repeat process
for i in np.arange(16):
    full_list = full_list + api.user_timeline(user_id=id,count=3200,exclude_replies=True,include_rts=False,max_id=currmax)
    currmax = full_list[len(full_list)-1].id
    print(currmax)
    full_list.pop()

print(len(full_list))

bodies = np.array([])
for tweet in full_list:
    bodies = np.append(bodies, tweet.text)

print(bodies)

1605995925339398145
1604230165051195394
1602803918790017025
1601020109807448064
1598157971384463360
1595682322707267584
1594131768298315777
1592228299337760768
1590383937284870145
1587498907336118274
1583299870370467840
1579934616429158400
1575508498430820352
1568364238451740674
1559766938498236416
1552324811153707018
479
['Some things that your party tells you are false and some things that the other party says are true'
 'All four orbital launch pads fully loaded with rockets for the first time! https://t.co/EXlB0p5vWN'
 'I hope that the people of Brazil are able to resolve matters peacefully'
 '(Chart by James Stephenson) https://t.co/rOHfjXGVmU'
 'Easy swipe right/left to move between recommended vs followed tweets rolls out later this week.\n\nFirst part of a mu… https://t.co/AGiPMijLK9'
 'Plaid is sickkk https://t.co/OxBD8Zv7Xq'
 'Thread on how safe Tesla cars are https://t.co/M3Hf3q4kh3'
 '8 years later &amp; still no laws … 😢 https://t.co/9fjb24XvFG'
 'If I dig my grave deep en

In [7]:
tweetbodies = pd.DataFrame()
tweetbodies = tweetbodies.assign(text=bodies)
tweetbodies = tweetbodies[tweetbodies.get("text").str.contains("https") == False].reset_index().drop(columns="index")
tweetbodies = tweetbodies[tweetbodies.get("text").str.len() >= 3]
tweetbodies

,text
0,Hope you’re having a great day 1 2023!\n\nOne ...
1,Sometimes it’s just better to make pizza at home
2,If you don’t think there’s at least a tiny cha...
3,New Twitter will strive to optimize unregrette...
4,Significant backend server architecture change...
...,...
203,What video games have you enjoyed most over pa...
204,Movie idea: Boss Baby meets Das Boot\nDas Baby...
205,Gehalt und Gestalt
206,Excited to work with Tesla Service to enable s...


In [8]:
def takefirsthalf(sentence):
    #try split by comma
    split=sentence.split(", ",1)

    #if split by comma doesn't work, split at 1/3
    if(len(split) == 1):
        split=sentence.split(" ")
        str = ""
        for i in np.arange((int)(len(split)/3)):
            str = str + split[i] + " "
        return str
    else:
        return split[0] + ","

def takenexthalf(sentence):
    #try split by comma
    split=sentence.split(", ",1)

    #if split by comma doesn't work, split at 1/3
    if(len(split) == 1):
        split=sentence.split(" ")
        str = ""
        for i in np.arange(len(split)-(int)(len(split)/3)):
            str = str + split[i+(int)(len(split)/3)] + " "
        return str
    else:
        return split[1]

tweetbodies = tweetbodies.assign(Prompt=tweetbodies.get("text").apply(takefirsthalf))
tweetbodies = tweetbodies.assign(Completion=tweetbodies.get("text").apply(takenexthalf))
tweetbodies = tweetbodies.drop(columns="text")
tweetbodies

,Prompt,Completion
0,Hope you’re having a great day 1 2023!\n\nOne ...,it won’t be boring.
1,Sometimes it’s just,better to make pizza at home
2,If you don’t think there’s,at least a tiny chance you’re an NPC … you’re ...
3,New Twitter,will strive to optimize unregretted user-minutes
4,Significant backend server,architecture changes rolled out. Twitter shoul...
...,...,...
203,What video games have,you enjoyed most over past year or two?
204,Movie idea: Boss Baby,meets Das Boot\nDas Baby – “Hijinks on the Hig...
205,Gehalt,und Gestalt
206,Excited to work with Tesla Service to,enable same-hour service as often as possible!...


In [9]:
tweetbodies.to_csv('tweets.csv',index=False)